# Analyze coverage of Grambank features
This code is mainly adapted from: https://github.com/WPoelman/typ-div-survey

In [5]:
import pandas as pd
import os


feature_url = 'https://raw.githubusercontent.com/WPoelman/typ-div-survey/c11c1c9aa0d7723655c6aaf047e7201a9d78ad03/data/gb_lang_feat_vals.csv'

df_gb_features = pd.read_csv(feature_url)

grambank_url = 'https://raw.githubusercontent.com/grambank/grambank/master/cldf/languages.csv'

df_gb_languages = pd.read_csv(grambank_url)

multiq_path = '../../../data/model_language_fidelity/Mistral-7B-Instruct-v0.1.csv'

df_multiq =  pd.read_csv(multiq_path)

In [25]:
print('Number of languages with corresponding code in grambank: ')
print(len(df_multiq.input_glottocode.unique()))


subset = df_gb_features[df_gb_features['Lang_ID'].isin(df_multiq.input_glottocode.unique()) ]

gb_feats  = [x for x in df_gb_features.columns if x.startswith("GB")]

GB_FEATURE_MISSING = {"no_cov", "?"}


def get_gb_feature_cov(gb, treat_as_missing):
    max_coverage_per_feature = dict()
    for i, (feature_name, series) in enumerate(gb.items()):
        if not feature_name.startswith("GB"):
            continue
        max_coverage_per_feature[feature_name] = {
            i for i in series.unique() if i not in treat_as_missing
        }
    return max_coverage_per_feature

gb_max_coverage = get_gb_feature_cov(df_gb_features, GB_FEATURE_MISSING)


# GB feature coverage
compl_cov, incompl_cov = [], []
for f_name in gb_feats:
    f_vals = {val for val in subset[f_name] if val not in GB_FEATURE_MISSING}
    if gb_max_coverage[f_name] == f_vals:
        compl_cov.append(f_name)
    else:
        incompl_cov.append(f_name)

coverage = len(compl_cov) / len(gb_feats)


print(coverage)

0.9538461538461539